In [ ]:
# Load libraries
import numpy as np
import pandas as pd

import pickle as pkl

In [ ]:
# Function for loading leaks
def load_leaks(leak_val):
    leak_dir = './time_series/stats/'
    
    train_leak_loc = leak_dir + 'train_leak_%s.csv'%leak_val
    train_leak = pd.read_csv(train_leak_loc).compiled_leak
    test_leak_loc = leak_dir + 'test_leak_%s.csv'%leak_val
    test_leak = pd.read_csv(test_leak_loc).compiled_leak
    
    return train_leak, test_leak

In [ ]:
# Load leaks
leak_val = 38
print '\nLoading train and test leaks...'
train_leak, test_leak = load_leaks(leak_val)
print 'Nonzero elements in train:', np.count_nonzero(train_leak)
print 'Nonzero elements in test:', np.count_nonzero(test_leak)

In [ ]:
!ls ../submissions/

In [ ]:
file_dir = '../submissions/'
files = [
    'tstest_cat_2v3_lag38_fullfeat_submit.csv',
    'tstest_lgb_2v2_lag38_fullfeat_submit.csv',
    'tstest_lgb_2v2_lag38_bad_submit_0.521.csv',
    'tstest_lgb_2v2_lag36_submit_0.52.csv',
    'ts_ensemble_v0.csv'
]
weights = [1.15, 1.15, 1, 0.5, 0.8]

In [ ]:
testidx_path = '../data/test_idx.pkl'
with open(testidx_path, 'rb') as handle:
    test_idx = pkl.load(handle)

In [ ]:
submissions = [pd.read_csv(file_dir+f) for f in files]

In [ ]:
submit_compile = np.zeros(test_idx.shape[0])
for i, df in enumerate(submissions):
    submit_compile += weights[i]*df['target'].values
submit_compile /= np.sum(weights)

In [ ]:
sub_df = pd.DataFrame()
sub_df['ID'] = test_idx
sub_df['target'] = submit_compile
sub_df.head(12)

In [ ]:
sub_name = '../submissions/ts_ensemble_v2.csv'
sub_df.to_csv(sub_name, index=False)

In [ ]:
pd.read_csv(sub_name).head()

In [ ]:
def root_mean_square_log_diff(sub1, sub2):
    return np.sqrt(np.mean(np.square(np.subtract(np.log1p(sub1), np.log1p(sub2)))))

def sum_abs_diff(sub1, sub2):
    return np.sum(np.abs(np.subtract(sub1, sub2)))

fri_df = pd.read_csv('../submissions/tstest_lgb_2v2_lag36_submit_0.52.csv')
sat_df = pd.read_csv('../submissions/tstest_lgb_2v2_lag38_bad_submit_0.52.csv')
sun_df = pd.read_csv('../submissions/tstest_lgb_2v2_lag38_good_submit.csv')

right_idx = np.where(test_leak!=0)[0]
wrong_idx = np.where(test_leak==0)[0]

sub_right = sub_df['target'].values[right_idx]
sun_right = sun_df['target'].values[right_idx]
fri_right = fri_df['target'].values[right_idx]
sat_right = sat_df['target'].values[right_idx]

sub_wrong = sub_df['target'].values[wrong_idx]
sun_wrong = sun_df['target'].values[wrong_idx]
fri_wrong = fri_df['target'].values[wrong_idx]
sat_wrong = sat_df['target'].values[wrong_idx]

In [ ]:
print 'Sum abs diff:'
print sum_abs_diff(sub_right, sun_right)
print sum_abs_diff(sub_wrong, sun_wrong)

print '\nRoot mean square log diff:'
print root_mean_square_log_diff(sub_right, sun_right)
print root_mean_square_log_diff(sub_wrong, sun_wrong)